In [7]:
import degirum as dg
from dg_eval_tools import *
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
from tqdm import tqdm

In [8]:
token = "<insert token here>"
model_name = "yolov8n_silu_coco--640x640_quant_tflite_edgetpu_1"

ground_truth_annotations_path = "http://100.102.102.108:8084/coco/val/labels.json"
image_folder_path = "http://100.102.102.108:8084/coco/val/data/"

In [9]:
zoo = dg.connect(dg.CLOUD, zoo_url="https://cs.degirum.com/degirum/ultralytics", token=token)
model = zoo.load_model(model_name)

In [10]:
# config parameters
config = get_config_json(model_name)
model.output_confidence_threshold = config["POST_PROCESS"]["OutputConfThreshold"]
model.output_nms_threshold = config["POST_PROCESS"]["OutputNMSThreshold"]
model.output_max_detections = config["POST_PROCESS"]["OutputMaxDetections"]
model.output_max_detections_per_class = config["POST_PROCESS"]["OutputMaxDetectionsPerClass"]
model.output_max_classes_per_detection = config["POST_PROCESS"]["OutputMaxClassesPerDetection"]
class_map = config["POST_PROCESS"]["ClassMap"]

In [11]:
with open(ground_truth_annotations_path) as annotations_file:
    anno = COCO(annotations_file)

files_dict = anno.dataset["images"]
path_list = [os.path.join(image_folder_path, f["file_name"]) for f in files_dict]

jdict=[]
with model:
    for image_number,predictions in tqdm(enumerate(model.predict_batch(path_list)), total=len(path_list)):
        image_id = files_dict[image_number]["id"]
        save_results_coco_json(predictions.results,jdict,image_id,class_map)

pred_json = model_name + "_predictions.json"
with open(pred_json, "w") as f:
    json.dump(jdict, f)


loading annotations into memory...
Done (t=1.65s)
creating index...
index created!


100%|██████████| 5000/5000 [04:54<00:00, 16.96it/s]


In [12]:
pred = anno.loadRes(pred_json)
eval_obj = COCOeval(anno, pred, "bbox")
eval_obj.params.imgIds = sorted([file["id"] for file in files_dict]) # image IDs to evaluate
eval_obj.evaluate()
eval_obj.accumulate()
eval_obj.summarize()

Loading and preparing results...
DONE (t=3.02s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=33.71s).
Accumulating evaluation results...
DONE (t=11.23s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.332
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.484
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.355
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.155
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.369
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.484
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.300
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.503
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.530
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=